I dati utilizzati in questo notebook sono stati presi dalla competizione di Kaggle [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic).

### Riferimenti bibliografici:

* Hastie, T.; Tibshirani, R. & Friedman, J. (2009), [The Elements of Statistical Learning](https://web.stanford.edu/~hastie/ElemStatLearn/).

# Alberi di decisione

## Indice

1. [Titanic: Machine Learning from Disaster](#titanic)<br>
    1.1 [Descrizione](#descrizione)<br>
    1.2 [Leggere i dati e dividere le variabili esplicative dalla variabile risposta](#leggere_dati)<br>
2. [Analisi esplorativa](#analisi_esplorativa)<br>
    2.1 [Studiare le variabili esplicative](#esplicative)<br>
    2.2 [Studiare la relazione tra variabili esplicative e variabile risposta](#esplicative_risposta)<br>
3. [Alberi di decisione](#alberi)<br>
    3.1 [Misure di impurità](#impurità)<br>
    3.2 [Creare una o più baseline](#baseline)<br>
4. [Scegliere gli iperparametri ottimali](#iperparametri)<br>
    4.1 [Definire la griglia di ricerca (*grid search*)](#grid)<br>
    4.2 [Calcolare l'accuratezza del modello per ogni combinazione degli iperparametri](#risultati)<br>
    4.3 [Visualizzare i risultati della ricerca](#visualizzare_risultati)<br>
    4.4 [Confrontare l'accuratezza del modello prima e dopo la scelta degli iperparametri ottimali](#confrontare)<br>
5. [Visualizzare le caratteristiche dell'albero](#visualizzare)<br>
    5.1 [Visualizzare l'albero](#visualizzare_albero)<br>
    5.2 [Visualizzare l'importanza delle variabili](#visualizzare_importanza)<br>
    5.3 [Visualizzare le superfici di decisione](#visualizzare_superfici)<br>

In [ ]:
import inspect
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

# 1. [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic) <a id=titanic> </a>

## 1.1 Descrizione <a id=descrizione> </a>

### Competition Description
The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

### Goal
It is your job to predict if a passenger survived the sinking of the Titanic or not. 
For each in the test set, you must predict a 0 or 1 value for the variable.

### Metric
Your score is the percentage of passengers you correctly predict. This is known simply as "accuracy”.

## 1.2 Leggere i dati e dividere le variabili esplicative dalla variabile risposta <a id=leggere_dati> </a>

### Leggere i dati

In [ ]:
PATH = "datasets/titanic"

dati = pd.read_csv(PATH + "/train.csv")
print("Dimensione del dataset: {} x {}".format(*dati.shape))
dati.head()

### Dividere le variabili esplicative dalla variabile risposta

In [ ]:
risposta =  "Survived"

X, y = dati.drop(columns=risposta).copy(), dati[risposta].copy()

# 2. Analisi esplorativa <a id=analisi_esplorativa> </a>

## 2.1 Studiare le variabili esplicative <a id=esplicative> </a>

In [ ]:
# TODO: vedi esercizio seguente

## 2.2 Studiare la relazione tra variabili esplicative e variabile risposta <a id=esplicative_risposta> </a>

### Dividiere i dati in training e validation

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
import seaborn as sns
import warnings

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

dati_grafico = X_train[["Age", "Fare"]].copy()
dati_grafico['Survived'] = y_train.values
dati_grafico.dropna(inplace=True)

g = sns.JointGrid("Age", "Fare", dati_grafico, ratio=2, height=6, space=0.2, ylim=[-10, 300])

for i, gr in dati_grafico.groupby("Survived"):
    sns.distplot(gr["Age"], ax=g.ax_marg_x)
    sns.distplot(gr["Fare"], ax=g.ax_marg_y, vertical=True)
    g.ax_joint.plot(gr["Age"], gr["Fare"], 'o', alpha=0.5)
plt.legend(["Morto", "Soprav"])
plt.show()

### Esercizio

Cosa si può dedurre dal grafico precedente?

In [ ]:
# TODO: vedi esercizio seguente

### Esercizio

1. Completare l'analisi esplorativa commentandone i risultati;
2. Creare almeno una variabile utilizzando le variabili "Name" e/o "Ticket" e/o "Cabin" sulla base dei risultati ottenuti al punto procedente.

### Eliminare le variabili che non si intendono utilizzare

> Nota: non avendo fatto l'analisi esplorativa, eliminiamo le variabili che richiederebbero un trattamento più lungo e una comprensione maggiore del data set. La riga seguente, come parti sucessive del notebook, vanno modificate a seconda delle variabili che si intende costruire sulla base dei risultati dell'analisi esplorativa.

In [ ]:
X = X.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
X_train = X_train.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
X_val = X_val.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
print("Variabili rimaste: {}.".format(", ".join(X.columns.tolist())))

# 3. Alberi di decisione <a id=alberi> </a>

## 3.1 Misure di impurità <a id=impurità> </a>

### Esercizio

Completare le funzioni `indice_gini` e `tasso_errata_classificazione` definite in `msbd/indici/indici.py`;

Si ricorda che:

$
I_G(p) = 2 p (1 - p)\\
I_{EC}(p) = 1 - \max(p, 1 - p)
$

per $p \in [0,1]$.

In [ ]:
from msbd.indici import indice_gini
from msbd.indici import tasso_errata_classificazione

print(inspect.getsource(indice_gini))
print(inspect.getsource(tasso_errata_classificazione))

In [ ]:
try:
    p = np.linspace(0, 1, 1000)

    plt.figure(figsize=(6, 3))

    plt.title("Misure di impurità")
    plt.plot(p, indice_gini(p), label="Indice di impurità di Gini")
    plt.plot(p, tasso_errata_classificazione(p), label="Tasso di errata classificazione")
    plt.xlabel("p")
    plt.legend()

    plt.show()
except:
    plt.close()
    print("Le funzioni indice_gini() e tasso_errata_classificazione() non sono implementate correttamente.")

### Esercizio

Dato un nodo $n_{00} = [\#\,\text{Morti}, \#\,\text{Sopravvissuti}] = [400, 400]$, calcolare l'indice di impurità di Gini e il tasso di errata classificazione per:
1. risultato della divisione $a$: $n^a_{10} = [300, 100]$ e $n^a_{11} = [100, 300]$;
2. Il risultato della divisione $b$: $n^b_{10} = [200, 400]$ e $n^b_{11} = [200, 0]$:

> Suggerimento: l'indice della divisione $a$ va calcolato come $I(a) = w^a_0 * I(n^a_{10}) + w^a_1 * I(n^a_{11})$ dove $w^a_0 = \frac{\sum n^a_{10}}{\sum n^a_{10} + \sum n^a_{11}}$ e $w^a_1 = \frac{\sum n^a_{11}}{\sum n^a_{10} + \sum n^a_{11}} = 1 - w^a_0$.

In [ ]:
n00 = np.array([400, 400])

n10_a = np.array([300, 100])
n11_a = np.array([100, 300])

n10_b = np.array([200, 400])
n11_b = np.array([200, 0])

try:
    # calcolare I_G(p) e I_EC(p) di n00, p = n00[1] / (n00[0] + n00[1])
    ig_n00 = indice_gini(n00[1] / n00.sum())
    ec_n00 = tasso_errata_classificazione(n00[1] / n00.sum())
except:
    ig_n00 = 0
    ec_n00 = 0
    print("Le funzioni indice_gini() e tasso_errata_classificazione() non sono implementate correttamente.\n")

# TODO: sostituire gli 0. con i valori corretti
# ============== YOUR CODE HERE ==============
ec_n10_a = 0.
ig_n11_a = 0.

ec_n10_b = 0.
ig_n11_b = 0.

ig_a = 0.
ec_a = 0.

ig_b = 0.
ec_b = 0.
# ============================================

print("Indice di impurità di Gini {}: {:.3f}".format(n00, ig_n00))
print("Tasso di errata classificazione {}: {:.3f}\n".format(n00, ig_n00))

print("Indice di impurità di Gini {} -> {} e {}: {:.3f}".format(n00, n10_a, n11_a, ig_a))
print("Tasso di errata classificazione {} -> {} e {}: {:.3f}\n".format(n00, n10_a, n11_a, ec_a))

print("Indice di impurità di Gini {} -> {} e {}: {:.3f}".format(n00, n10_b, n11_b, ig_b))
print("Tasso di errata classificazione {} -> {} e {}: {:.3f}\n".format(n00, n10_b, n11_b, ec_b))

## 3.2 Creare una o più baseline <a id=baseline> </a>

### `DummyClassifier`

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dc = DummyClassifier("most_frequent")

dc.fit(X_train, y_train)

y_pred = dc.predict(X_val)
dc_acc = accuracy_score(y_val, y_pred)

print('Accuratezza DummyClassifier("most_frequent"): {:.1f}%'.format(100 * dc_acc))

### `DecisionTreeClassifier` (senza ottimizzazione degli iperparametri)

In [ ]:
from msbd.preprocessamento import OttenereDummy
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = Pipeline([
    ("ottenere_dummy", OttenereDummy(drop_first=True)),
    ("imputer", SimpleImputer(strategy="mean")), 
    ("tree", DecisionTreeClassifier())
])

dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_val)
dtc_acc = accuracy_score(y_val, y_pred)

print('Accuratezza DummyClassifier("most_frequent"): {:.1f}%'.format(100 * dc_acc))
print("Accuratezza DecisionTreeClassifier(): {:.1f}%".format(100 * dtc_acc))

### Esercizio

Calcolare l'accuratezza sull'insieme di *training*. Ci sono segnali di sovradattamento?

# 4. Scegliere gli iperparametri ottimali <a id=iperparametri> </a>

## 4.1 Definire la griglia di ricerca (*grid search*) <a id=grid> </a>

In [ ]:
print("Profondità dell'albero allenato senza restrizioni: {}".format(dtc.named_steps["tree"].tree_.max_depth))
print("Massimo numero minimo di osservazioni in una foglia: {}".format(len(X_train) // 2))

In [ ]:
from sklearn.model_selection import ParameterGrid
import tqdm

In [ ]:
param_grid = ParameterGrid({
    'tree__max_depth': np.arange(1, 18),
    'tree__min_samples_leaf': 2 ** np.arange(9),
})
print(param_grid.param_grid)

> Nota: la ricerca degli iperparametri ottimali tramite *grid search* è fattibile solo quando questi sono pochi. Nel caso in cui gli iperparameteri siano molti, un'approccio migliore è  la *random search*. L'equivalente di [ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) per la *random search* è [ParameterSampler](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterSampler.html).

![search](figures/search.png)

*Immagine presa da* [Random Search for Hyper-Parameter Optimization](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf).

### Esercizio

Immaginirare di dover trovare i valori ottimali di 10 iperparameteri differenti. Per ogniuno di essi vogliamo scegliere tra 5 possibili valori. Calcolare quante combinazioni si devono valutare (e quindi quante volte si deve allenare il modello) con il metodo *grid search* e con il metodo *random search*.

## 4.2 Calcolare l'accuratezza del modello per ogni combinazione degli iperparametri <a id=risultati> </a>

In [ ]:
risultati = []

for params in tqdm.tqdm(param_grid):
    dtc.set_params(**params)
    dtc.fit(X_train, y_train)
    y_pred = dtc.predict(X_val)
    params["accuracy_score"] = accuracy_score(y_val, y_pred)
    risultati.append(params)

risultati = pd.DataFrame(risultati).sort_values(["accuracy_score", "tree__max_depth"], ascending=[False, True])
risultati.reset_index(drop=True, inplace=True)
print("Primi 5:")
display(risultati.head())

print("Ultimi 5:")
risultati.tail()

## 4.3 Visualizzare i risultati della ricerca <a id=visualizzare_risultati> </a>

In [ ]:
from msbd.grafici import grafico_metrica_iperparametro

print(inspect.getsource(grafico_metrica_iperparametro))

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(121)
grafico_metrica_iperparametro(risultati, "tree__max_depth", "accuracy_score", alpha=0.5)

plt.subplot(122)
grafico_metrica_iperparametro(risultati, "tree__min_samples_leaf", "accuracy_score", alpha=0.5)
plt.xscale("log", basex=2)

plt.show()

In [ ]:
from msbd.grafici import grafico_metrica_iperparametri

print(inspect.getsource(grafico_metrica_iperparametri))

In [ ]:
plt.figure(figsize=(12, 6))

grafico_metrica_iperparametri(risultati, "tree__max_depth", "tree__min_samples_leaf", "accuracy_score")
plt.yscale("log", basey=2)

plt.show()

## 4.4 Confrontare l'accuratezza del modello prima e dopo la scelta degli iperparametri ottimali <a id=confrontare> </a>

In [ ]:
max_depth = risultati.loc[0, "tree__max_depth"]
min_samples_leaf = risultati.loc[0, "tree__min_samples_leaf"]

dtc_tun = Pipeline([
    ("ottenere_dummy", OttenereDummy(drop_first=True)),
    ("imputer", SimpleImputer(strategy="mean")), 
    ("tree", DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf))
])

dtc_tun.fit(X_train, y_train)

y_pred = dtc_tun.predict(X_val)
dtc_tun_acc = accuracy_score(y_val, y_pred)

print('Accuratezza DummyClassifier("most_frequent"): {:.1f}%'.format(100 * dc_acc))
print("Accuratezza DecisionTreeClassifier(): {:.1f}%".format(100 * dtc_acc))
print("Accuratezza DecisionTreeClassifier(max_depth={}, min_samples_leaf={}): {:.1f}%".format(
    max_depth, min_samples_leaf, 100 * dtc_tun_acc))

# 5. Visualizzare le caratteristiche dell'albero <a id=visualizzare> </a>

## 5.1 Visualizzare l'albero <a id=visualizzare_albero> </a>

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

In [ ]:
dot_data = export_graphviz(
    decision_tree=dtc_tun.named_steps["tree"], 
    max_depth=4,
    feature_names=dtc.named_steps["ottenere_dummy"].columns_,
    class_names=("Morto", "Soprav"),
    filled=True,
    rounded=True,
)
display(graphviz.Source(dot_data))

## 5.2 Visualizzare l'importanza delle variabili <a id=visualizzare_importanza> </a>

In [ ]:
from msbd.grafici import grafico_importanza_variabili

print(inspect.getsource(grafico_importanza_variabili))

In [ ]:
importanze = dtc_tun.named_steps["tree"].feature_importances_
variabili = dtc_tun.named_steps["ottenere_dummy"].columns_

grafico_importanza_variabili(importanze, variabili)

plt.show()

## 5.3 Visualizzare le superfici di decisione <a id=visualizzare_superfici> </a>

### Esercizio

Imitare, per le variabili per cui ha senso, i grafici presenti in [Plot the decision surface of a decision tree on the iris dataset](https://scikit-learn.org/stable/auto_examples/tree/plot_iris.html).